In [18]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error


def mackey_glass(T, tau=17, n=10, beta=0.2, gamma=0.1):
    x = np.zeros(T)
    x[0] = 1.2
    for t in range(1, T):
        if t - tau >= 0:
            x[t] = x[t-1] + beta * x[t-tau] / (1 + x[t-tau]**n) - gamma * x[t-1]
        else:
            x[t] = x[t-1]
    return x

class EchoStateNetwork:
    def __init__(self, input_dim, reservoir_size, output_dim, spectral_radius=0.9, alpha=1e-6):
        self.input_dim = input_dim
        self.reservoir_size = reservoir_size
        self.output_dim = output_dim
        self.spectral_radius = spectral_radius
        self.alpha = alpha

        self.Win = torch.rand((reservoir_size, input_dim)) * 2 - 1
        self.W = torch.rand((reservoir_size, reservoir_size)) * 2 - 1
        self.adjust_spectral_radius()
        
        self.readout = Ridge(alpha=self.alpha)
    
    def initialize_weights(self):
        self.Win = torch.rand((self.reservoir_size, self.input_dim)) * 2 - 1
        self.W = torch.rand((self.reservoir_size, self.reservoir_size)) * 2 - 1
        self.adjust_spectral_radius()

    def adjust_spectral_radius(self):
        eigenvalues, _ = torch.linalg.eig(self.W)
        rhoW = max(abs(eigenvalues))
        self.W *= self.spectral_radius / rhoW

def train_esn(model, train_data, target_data, num_trials=50):
    results = []
    for trial in range(num_trials):
        torch.manual_seed(trial)
        np.random.seed(trial)  # Ensure NumPy is also seeded
        model.initialize_weights()

        state = torch.zeros(model.reservoir_size)
        states = []
        
        for t in range(len(train_data)):
            u = torch.tensor([train_data[t]], dtype=torch.float32)
            state = torch.tanh(model.Win @ u + model.W @ state)
            states.append(state.numpy())
        
        states = np.array(states)
        model.readout.fit(states, target_data)
        predictions = model.readout.predict(states)
        mse = mean_squared_error(target_data, predictions)
        results.append(mse)
    
        print(f'Trial {trial + 1}/{num_trials}: MSE = {mse}')
    return mse, predictions

In [19]:
# Generate Mackey-Glass time series data
T = 1000
data = mackey_glass(T)
train_data, target_data = data[:800], data[200:1000]

neurons = 100  # Number of neurons in the reservoir
num_trials = 50  # Number of trials with different initializations
input_dim = 1
output_dim = 1
esn = EchoStateNetwork(input_dim, neurons, output_dim)

mse, predictions = train_esn(esn, train_data, target_data, num_trials)
print(f'Average MSE over {num_trials} trials: {np.mean(mse)}')

Trial 1/50: MSE = 0.014141706764518256
Trial 2/50: MSE = 0.013283701805913673
Trial 3/50: MSE = 0.013635773117935629
Trial 4/50: MSE = 0.013601536545094457
Trial 5/50: MSE = 0.013505257087724144
Trial 6/50: MSE = 0.013938276885673872
Trial 7/50: MSE = 0.01390492908456615
Trial 8/50: MSE = 0.013587816414938954
Trial 9/50: MSE = 0.013705599141463763
Trial 10/50: MSE = 0.013817790392645102
Trial 11/50: MSE = 0.013884169989002099
Trial 12/50: MSE = 0.014245300759134185
Trial 13/50: MSE = 0.013697304834924772
Trial 14/50: MSE = 0.013745844463027148
Trial 15/50: MSE = 0.013897264151519846
Trial 16/50: MSE = 0.01419021326450797
Trial 17/50: MSE = 0.013240856621552326
Trial 18/50: MSE = 0.013204069807635024
Trial 19/50: MSE = 0.013799337144882693
Trial 20/50: MSE = 0.014020511406810704
Trial 21/50: MSE = 0.013071196368526397
Trial 22/50: MSE = 0.013661247161480863
Trial 23/50: MSE = 0.013543381758312333
Trial 24/50: MSE = 0.014064607895272939
Trial 25/50: MSE = 0.013919030315726175
Trial 26/50